# Database loader
This notebook provides a step-by-step guide for loading data into an ArangoDB database.

In [ ]:
# 1. Install dependencies
#! pip install python-arango --upgrade

In [3]:
# 2. Import modules
import os
import json
import sys
from arango import ArangoClient, DocumentInsertError
from streamlit_arango.config import Config


In [4]:
# 3. Define file paths
db_dir = '../output'
db_nodes_path = os.path.join(db_dir, "fermentdb_nodes_full.json")
db_edges_path = os.path.join(db_dir, "fermentdb_edges_full.json")

### 1. Load data: nodes and edges dictionaries. 

In [5]:
# Load nodes json
with open(db_nodes_path, 'r') as dbfile:
    collections_str = dbfile.read()

nodes = json.loads(collections_str)

In [6]:
# Load edges json
with open(db_edges_path, 'r') as dbfile:
    edges_str = dbfile.read()

edges = json.loads(edges_str)

### 2. Connect to ArangoDB

In [7]:
# Initialize the client for ArangoDB.
client = ArangoClient(hosts=Config.ArangoDB.host)
# client = ArangoClient(hosts="http://localhost:8529")

# Connect to "_system" database as root user.
sys_db = client.db("_system", username=Config.ArangoDB.username, password= Config.ArangoDB.password)
# sys_db = client.db("_system", username="root", password= "")

# Create a new database named "fermentdb" if it does not exist.
if not sys_db.has_database(Config.ArangoDB.database):
    sys_db.create_database(Config.ArangoDB.database)

# Retrieve the names of all databases on the server as list of strings
db_list = sys_db.databases()
print(db_list)

['_system', 'fermentdb']


### 3. Connect to FermentDB and create collections: nodes and edges

In [8]:
# Connect to "fermentdb" database as root user.
db = client.db(Config.ArangoDB.database, username=Config.ArangoDB.username, password=Config.ArangoDB.password)
# db = client.db("fermentdb", username="root", password="")

In [14]:
# Create graph fermentdb if it does not exist.
if db.has_graph(Config.ArangoDB.database):
    db.delete_graph(Config.ArangoDB.database, drop_collections=True)
    db.create_graph(Config.ArangoDB.database)
else:
    db.create_graph(Config.ArangoDB.database)

fermentdb = db.graph(Config.ArangoDB.database)

In [15]:
# Check graph
db.graphs()
print("----")
fermentdb.name
fermentdb.db_name
fermentdb.vertex_collections()
print("----")
fermentdb.edge_definitions()

----
----


[]

In [ ]:
# Load node/vertex collections
for node in nodes:
    if fermentdb.has_vertex_collection(node):
        v = fermentdb.vertex_collection(node)
    else:
        v = fermentdb.create_vertex_collection(node)

    if type(nodes[node]) != list:
        nodes[node] = [nodes[node]]
    
    v.insert_many(nodes[node])

# List vertex collections in the graph.
fermentdb.vertex_collections()

In [ ]:
# Load edges collections
missing_docs = []
for edge in edges:
    if fermentdb.has_edge_definition(edge):
        e = fermentdb.edge_collection(edge)
    else:
        e = fermentdb.create_edge_definition(
            edge_collection=edge,
            from_vertex_collections=edges[edge]['from_collection'],
            to_vertex_collections=edges[edge]['to_collection']
        )
        
    # if not fermentdb.has_edge_definition(edge):
    #     e = fermentdb.create_edge_definition(
    #         edge_collection=edge,
    #         from_vertex_collections=edges[edge]['from_collection'],
    #         to_vertex_collections=edges[edge]['to_collection']
    #     )

    for l in edges[edge]['edges']:
        try:
            e.insert(l)
        except DocumentInsertError as exc:
            missing_docs.append(l)
    
    # List edge definitions.
fermentdb.edge_definitions()